<a href="https://colab.research.google.com/github/AG-2002/Customer-Segmentation-and-Prediction-of-Term-Deposit-Subscriptions-in-Bank-Marketing/blob/main/Bank_Data(Model_fitting(Boosting_Models))_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('bank_data_cleaned.csv')
df

,age,month,day_of_week,duration,campaign,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,...,housing_unknown,housing_yes,loan_unknown,loan_yes,contact_telephone,poutcome_nonexistent,poutcome_success,pdays_new_recently contacted,pdays_new_sometimes ago,y
0,56,5,1,261,1,0,1.1,93.994,-36.4,4.857,...,0,0,0,0,1,1,0,0,0,0
1,57,5,1,149,1,0,1.1,93.994,-36.4,4.857,...,0,0,0,0,1,1,0,0,0,0
2,37,5,1,226,1,0,1.1,93.994,-36.4,4.857,...,0,1,0,0,1,1,0,0,0,0
3,40,5,1,151,1,0,1.1,93.994,-36.4,4.857,...,0,0,0,0,1,1,0,0,0,0
4,56,5,1,307,1,0,1.1,93.994,-36.4,4.857,...,0,0,0,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41171,73,11,5,334,1,0,-1.1,94.767,-50.8,1.028,...,0,1,0,0,0,1,0,0,0,1
41172,46,11,5,383,1,0,-1.1,94.767,-50.8,1.028,...,0,0,0,0,0,1,0,0,0,0
41173,56,11,5,189,2,0,-1.1,94.767,-50.8,1.028,...,0,1,0,0,0,1,0,0,0,0
41174,44,11,5,442,1,0,-1.1,94.767,-50.8,1.028,...,0,0,0,0,0,1,0,0,0,1


# **----------  ADA BOOST  ----------**

In [3]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 13.4 MB/s eta 0:00:00


In [11]:
import optuna
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.metrics import classification_report, accuracy_score,confusion_matrix, recall_score, precision_score
from sklearn.model_selection import cross_val_score

In [5]:
X = df.drop(columns='y')
y = df['y']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale features
scaler = StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
base_estimator = DecisionTreeClassifier(max_depth=1)

ada_model = AdaBoostClassifier(
    estimator=base_estimator,
    n_estimators=100,
    learning_rate=1.0,
    random_state=42
)

ada_model.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   n_estimators=100, random_state=42)

In [ ]:
# Predict on test data
y_pred = ada_model.predict(X_test)

# Accuracy and full classification report
print(" Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

 Accuracy: 0.9048081593006314
[[7096  212]
 [ 572  356]]

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.97      0.95      7308
           1       0.63      0.38      0.48       928

    accuracy                           0.90      8236
   macro avg       0.78      0.68      0.71      8236
weighted avg       0.89      0.90      0.89      8236



In [ ]:
def objective_adaboost(trial):

    n_estimators = trial.suggest_int('n_estimators', 10, 100)
    warm_start = trial.suggest_categorical('warm_start', [True, False])


    model = EasyEnsembleClassifier(
        n_estimators=n_estimators,
        warm_start=warm_start,
        random_state=42
    )

    score = cross_val_score(model, X_train, y_train, cv=5, scoring='recall_macro').mean()
    return score

In [ ]:
study_adaboost= optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study_adaboost.optimize(objective_adaboost, n_trials=50)

print(f'Best trial accuracy: {study_adaboost.best_trial.value}')
print(f'Best hyperparameters: {study_adaboost.best_trial.params}')

[I 2025-07-28 17:24:14,069] A new study created in memory with name: no-name-1d1f5f22-cab1-4056-9c01-7c2a8828bc96
[I 2025-07-28 17:25:27,584] Trial 0 finished with value: 0.8703290453584198 and parameters: {'n_estimators': 34, 'warm_start': True}. Best is trial 0 with value: 0.8703290453584198.
[I 2025-07-28 17:26:47,100] Trial 1 finished with value: 0.8707673909925857 and parameters: {'n_estimators': 37, 'warm_start': False}. Best is trial 1 with value: 0.8707673909925857.
[I 2025-07-28 17:29:27,816] Trial 2 finished with value: 0.8716231809089268 and parameters: {'n_estimators': 76, 'warm_start': True}. Best is trial 2 with value: 0.8716231809089268.
[I 2025-07-28 17:32:07,400] Trial 3 finished with value: 0.871505700046771 and parameters: {'n_estimators': 75, 'warm_start': True}. Best is trial 2 with value: 0.8716231809089268.
[I 2025-07-28 17:34:16,073] Trial 4 finished with value: 0.8712850407423588 and parameters: {'n_estimators': 60, 'warm_start': False}. Best is trial 2 with va

Best trial accuracy: 0.871640105234766
Best hyperparameters: {'n_estimators': 63, 'warm_start': False}


In [ ]:
best_model_adaboost = EasyEnsembleClassifier(n_estimators=27, random_state=42,warm_start=False)
best_model_adaboost.fit(X_train, y_train)
y_pred = best_model_adaboost.predict(X_test)

In [ ]:
# Accuracy and full classification report
print(" Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

 Accuracy: 0.8590335114133074
[[6236 1072]
 [  89  839]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.85      0.91      7308
           1       0.44      0.90      0.59       928

    accuracy                           0.86      8236
   macro avg       0.71      0.88      0.75      8236
weighted avg       0.92      0.86      0.88      8236



# **----------  GRADIENT BOOSTING  ----------**

In [6]:
from sklearn.ensemble import GradientBoostingClassifier

In [7]:
grad_clf = EasyEnsembleClassifier(estimator=GradientBoostingClassifier())
grad_clf.fit(X_train, y_train)

EasyEnsembleClassifier(estimator=GradientBoostingClassifier())

In [8]:
# Predict on test data
y_pred = grad_clf.predict(X_test)

# Accuracy and full classification report
print(" Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

 Accuracy: 0.8547838756677999
[[6165 1143]
 [  53  875]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.84      0.91      7308
           1       0.43      0.94      0.59       928

    accuracy                           0.85      8236
   macro avg       0.71      0.89      0.75      8236
weighted avg       0.93      0.85      0.88      8236



In [13]:
def objective_Gboost(trial):

    loss = trial.suggest_categorical('loss', ['log_loss', 'exponential'])
    learning_rate = trial.suggest_float('learning_rate', 0.001, 1)
    gboost_n_estimators  = trial.suggest_int('gboost_n_estimators', 50, 500)
    criterion = trial.suggest_categorical('criterion', ['friedman_mse', 'squared_error'])
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    max_features = trial.suggest_categorical('max_features', [None, 'sqrt', 'log2'])

    n_estimators = trial.suggest_int('n_estimators', 10, 100)
    warm_start = trial.suggest_categorical('warm_start', [True, False])


    model = EasyEnsembleClassifier(
        estimator=GradientBoostingClassifier(
            loss=loss,
            learning_rate=learning_rate,
            n_estimators=gboost_n_estimators,
            criterion=criterion,
            min_samples_split=min_samples_split,
            max_features=max_features
        ),
        n_estimators=n_estimators,
        warm_start=warm_start,
        random_state=42
    )

    score = cross_validate(model, X_train, y_train, cv=5, scoring=['recall_macro', 'f1_macro', 'precision_macro'], return_train_score=False)
    return score['test_recall_macro'].mean()

In [14]:
study_Gboost = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study_Gboost.optimize(objective_Gboost, n_trials=10)

print(f'Best trial accuracy: {study_Gboost.best_trial.value}')
print(f'Best hyperparameters: {study_Gboost.best_trial.params}')

[I 2025-07-29 17:59:01,600] A new study created in memory with name: no-name-f1c15d86-7669-45f7-bd3e-3f9d4f5ab30a
[I 2025-07-29 18:00:10,854] Trial 0 finished with value: 0.8866576711928816 and parameters: {'loss': 'exponential', 'learning_rate': 0.6553325423013676, 'gboost_n_estimators': 56, 'criterion': 'friedman_mse', 'min_samples_split': 3, 'max_features': 'log2', 'n_estimators': 54, 'warm_start': True}. Best is trial 0 with value: 0.8866576711928816.
[I 2025-07-29 18:09:32,719] Trial 1 finished with value: 0.8900241060783006 and parameters: {'loss': 'exponential', 'learning_rate': 0.7440641248248181, 'gboost_n_estimators': 427, 'criterion': 'friedman_mse', 'min_samples_split': 8, 'max_features': 'log2', 'n_estimators': 63, 'warm_start': True}. Best is trial 1 with value: 0.8900241060783006.
[I 2025-07-29 18:23:04,953] Trial 2 finished with value: 0.8914094885290895 and parameters: {'loss': 'exponential', 'learning_rate': 0.28910152077131795, 'gboost_n_estimators': 206, 'criterion'

Best trial accuracy: 0.8920017446112614
Best hyperparameters: {'loss': 'log_loss', 'learning_rate': 0.7226895503824824, 'gboost_n_estimators': 180, 'criterion': 'squared_error', 'min_samples_split': 7, 'max_features': None, 'n_estimators': 94, 'warm_start': False}


In [17]:
params_gboost = study_Gboost.best_params

gboost_params = {
    'loss': params_gboost['loss'],
    'learning_rate': params_gboost['learning_rate'],
    'n_estimators': params_gboost['gboost_n_estimators'],
    'criterion': params_gboost['criterion'],
    'min_samples_split': params_gboost['min_samples_split'],
    'max_features': params_gboost['max_features']
}

ensemble_params = {
    'n_estimators': params_gboost['n_estimators'],
    'warm_start': params_gboost['warm_start'],
    'random_state': 42
}
best_model_Gboost = EasyEnsembleClassifier(
    estimator=GradientBoostingClassifier(**gboost_params),
    **ensemble_params)
best_model_Gboost.fit(X_train, y_train)
y_pred_Gboost_optuna = best_model_Gboost.predict(X_test)

In [20]:
# Predict on test data
y_pred = best_model_Gboost.predict(X_test)

# Accuracy and full classification report
print(" Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

 Accuracy: 0.8641330743079164
[[6238 1070]
 [  49  879]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.85      0.92      7308
           1       0.45      0.95      0.61       928

    accuracy                           0.86      8236
   macro avg       0.72      0.90      0.76      8236
weighted avg       0.93      0.86      0.88      8236



# **----------  XGBoost  ----------**

In [21]:
from xgboost import XGBClassifier

In [22]:
xgbc = XGBClassifier(n_estimators=100,  max_depth=4, learning_rate=0.1, gamma=1, reg_lambda=1, reg_alpha=0)
xgbc.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=1, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, ...)

In [23]:
# Predict on test data
y_pred = xgbc.predict(X_test)

# Accuracy and full classification report
print(" Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

 Accuracy: 0.9181641573579408
[[7054  254]
 [ 420  508]]

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.97      0.95      7308
           1       0.67      0.55      0.60       928

    accuracy                           0.92      8236
   macro avg       0.81      0.76      0.78      8236
weighted avg       0.91      0.92      0.91      8236



In [24]:
def objective_XGboost(trial):

    n_estimators_xgboost = trial.suggest_int('n_estimators_xgboost', 100, 500)
    max_depth = trial.suggest_int('max_depth', 2, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 1)
    gamma = trial.suggest_float('gamma', 0, 1)
    reg_lambda = trial.suggest_float('reg_lambda', 0, 1)
    reg_alpha = trial.suggest_float('reg_alpha', 0, 1)

    n_estimators = trial.suggest_int('n_estimators', 10, 100)
    warm_start = trial.suggest_categorical('warm_start', [True, False])

    model = EasyEnsembleClassifier(
        estimator=XGBClassifier(
            n_estimators=n_estimators_xgboost,
            max_depth=max_depth,
            learning_rate=learning_rate,
            gamma=gamma,
            reg_lambda=reg_lambda,
            reg_alpha=reg_alpha,
            ),
        n_estimators = trial.suggest_int('n_estimators', 10, 100),
        warm_start = trial.suggest_categorical('warm_start', [True, False])
        )

    score = cross_validate(model, X_train, y_train, cv=5, scoring=['recall_macro', 'f1_macro', 'precision_macro'], return_train_score=False)
    return score['test_recall_macro'].mean()

In [25]:
study_XGboost = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study_XGboost.optimize(objective_XGboost, n_trials=50)

print(f'Best trial accuracy: {study_XGboost.best_trial.value}')
print(f'Best hyperparameters: {study_XGboost.best_trial.params}')

[I 2025-07-29 19:29:50,174] A new study created in memory with name: no-name-4157c5f0-8775-4e4b-b814-23e3292cf76a
[I 2025-07-29 19:31:01,456] Trial 0 finished with value: 0.8914119915065692 and parameters: {'n_estimators_xgboost': 183, 'max_depth': 3, 'learning_rate': 0.7728267823513818, 'gamma': 0.5634383176642095, 'reg_lambda': 0.7031441075988655, 'reg_alpha': 0.9662371864762711, 'n_estimators': 80, 'warm_start': False}. Best is trial 0 with value: 0.8914119915065692.
[I 2025-07-29 19:31:42,015] Trial 1 finished with value: 0.8946863686082194 and parameters: {'n_estimators_xgboost': 211, 'max_depth': 10, 'learning_rate': 0.5106749641281461, 'gamma': 0.2156770760339377, 'reg_lambda': 0.555898187959403, 'reg_alpha': 0.8111096643592304, 'n_estimators': 34, 'warm_start': True}. Best is trial 1 with value: 0.8946863686082194.
[I 2025-07-29 19:32:46,749] Trial 2 finished with value: 0.8934232339798216 and parameters: {'n_estimators_xgboost': 156, 'max_depth': 7, 'learning_rate': 0.22360233

Best trial accuracy: 0.8958951168727942
Best hyperparameters: {'n_estimators_xgboost': 230, 'max_depth': 5, 'learning_rate': 0.8659018322821411, 'gamma': 0.4458315916766231, 'reg_lambda': 0.4316964052125021, 'reg_alpha': 0.8700086946100859, 'n_estimators': 93, 'warm_start': True}


In [26]:
params_xgboost = study_XGboost.best_params

xgb_params = {
    'n_estimators': params_xgboost['n_estimators_xgboost'],
    'max_depth': params_xgboost['max_depth'],
    'learning_rate': params_xgboost['learning_rate'],
    'gamma': params_xgboost['gamma'],
    'reg_lambda': params_xgboost['reg_lambda'],
    'reg_alpha': params_xgboost['reg_alpha'],
    'random_state': 42
}

ensemble_params = {
    'n_estimators': params_xgboost['n_estimators'],
    'warm_start': params_xgboost['warm_start'],
    'random_state': 42
}

best_model_XGboost = EasyEnsembleClassifier(
    estimator=XGBClassifier(**xgb_params),
    **ensemble_params
)

best_model_XGboost.fit(X_train, y_train)
y2_pred_XGboost_optuna = best_model_XGboost.predict(X_test)

In [27]:
# Predict on test data
y_pred = best_model_XGboost.predict(X_test)

# Accuracy and full classification report
print(" Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

 Accuracy: 0.8654686741136474
[[6246 1062]
 [  46  882]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.85      0.92      7308
           1       0.45      0.95      0.61       928

    accuracy                           0.87      8236
   macro avg       0.72      0.90      0.77      8236
weighted avg       0.93      0.87      0.88      8236

